<a href="https://colab.research.google.com/github/vicente-gonzalez-ruiz/MCDWT-experimentos/blob/master/src/MDWT_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [MDWT](https://sistemas-multimedia.github.io/PRMC/#x1-100005)$(N=5)$
Calculamos la transformada directa e inversa de una iteración, a 5 imágenes. El objetivo principal de este experimento es mostrar la reversibilidad de la DWT proporcionada por [PyWavelets](https://pywavelets.readthedocs.io/en/latest/).

In [ ]:
%matplotlib inline
from load_and_display import *

# The results depends on the wavelet (see http://wavelets.pybytes.com):
#wavelet = "haar"
wavelet = "bior3.5"
#wavelet = "rbio3.5"
#wavelet = "db5"
#wavelet = "db9"

 El vídeo usado es:

In [ ]:
show_seq("../sequences/stockholm/")

Los vídeos con los frames se almacenan en disco como imágnes PNG de 16 bits en el rango [0, 65535]. Esta tipo de formato se ha usado también para representar los coeficientes wavelet. Como estos pueden ser negativos, antes de ser almacenados a disco hay que sumarles 32768, y rezar para que su rango dinámico no exceda $2^{16}$ (cosa que rara vez ocurre si las imágenes de partida vienen a 8 bits/componente). Por tanto, tras leerlos de disco hay que restarles este valor. También ocurre que para visualizar los coeficientes hay que normalizarlos, porque no tienen por qué estar dentro del rango [0, 255]. El espacio de color usado para los frames es RGB, aunque en algunas fases del procesamiento haya que usar YCbCr. 

También podemos ver el movimiento en la secuencia restando imágenes consecutivas:

In [ ]:
show_motion("../sequences/stockholm/")

## $\mathtt{MDWT}(N=5)$

Aplicamos la 2D-DWT una iteración a cada una de las imágenes de la secuencia.

In [ ]:
%%bash

# You must be in the ’src’ directory to run this.
rm -f /tmp/*.png  
cp ../sequences/stockholm/*.png /tmp  
 
# 1-iteration MDWT.  
python3 -O MDWT.py -p /tmp/  
 
# Show the length of the subbands.  
for i in /tmp/LL???.png; do ls -lh $i; done  
for i in /tmp/LH???.png; do ls -lh $i; done  
for i in /tmp/HL???.png; do ls -lh $i; done  
for i in /tmp/HH???.png; do ls -lh $i; done  

### Visualizamos la subbanda espacial $L$

La subbanda $L$ está compuesta por 5 subbandas espaciales $LL^1$ con una resolución mitad (en cada dimensión) de la original. [Aquí](https://sistemas-multimedia.github.io/PRMC/#x1-220009.5) se habla de la notación seguida para referenciar a las subbandas.

In [ ]:
show_seq("/tmp/LL")

In [ ]:
show_motion("/tmp/LL")

### Visualizamos la subbanda $H=\{LH, HL, HH\}$

In [ ]:
show_seq("/tmp/LH")

In [ ]:
show_seq("/tmp/HL")

In [ ]:
show_seq("/tmp/HH")

## Inverse $\mathtt{MDWT(N=5)}$


In [ ]:
%%bash
 
# 1-iteration iMDWT.  
python3 -O MDWT.py -b -p /tmp/  

Mostramos la secuencia reconstruída:

In [ ]:
show_seq("/tmp/")

Mostramos algunas estadísticas de la secuencia original y de la reconstruída:

In [ ]:
%%bash 
 
# Show some statistics.
for i in {0..4}; do
  ii=$(printf "%03d" $i);
  python ../tools/show_statistics.py -i ../sequences/stockholm/$ii.png;
  python ../tools/show_statistics.py -i /tmp/$ii.png;
done  

Las imágenes reconstruídas son parecidas a las originales, pero no idénticas. Quizás el dato más relevante de la reconstrucción es que aumenta ligeramente el rango dinámico (de 255 pasamos a 258). También aumenta un poco la entropía. En principio, el origen de dicha distorsión es la aritmética en punto flotante usada para calcular la DWT.

Finalmente, mostramos las diferencia entre las secuencias original y reconstruída:

In [ ]:
%%bash 

# Visualization of the residue.  
for i in {0..4}; do ii=$(printf "%03d" $i); bash ../tools/show_differences.sh -1 /tmp/$ii.png -2 ../sequences/stockholm/$ii.png -o /tmp/diff_$ii.png; done  

In [ ]:
show_seq("/tmp/diff_")